In [221]:
# CMPS140 Project - How to Become Top Earner in Airbnb?
# Tangni Wang, Tung Hoi Man, Yunxiang Fu
# {twang63, tuman, yfu7}@ucsc.edu

In [222]:
# To do:

# Split Training set, testing set in code?
# Convert attributes to numbers
# Generate graph
# Different model with different functions? Which function do we use?


In [223]:
# Helper functions
def to_int(x):
    '''
    Columns value corrections
    '''
    if x=='f':
        x=x.replace('f','0')
    elif x=='t':
        x=x.replace('t','1')
    else:
        x= '0'
    return int(x)

def changeTime(x):
    '''
    change host_response_time columns from string into numerical.
    '''
    if x == 'within an hour':
        x='1'
    elif x == 'within a few hours':
        x='4'
    elif x == 'within a day':
        x='24'
    elif x == 'a few days or more':
        x='48'
    else:
        x='96'
        
    return x


def changeStr(x):
    '''
    change back the host_response_time from the numerical into strings
    '''
    if x == 1:
        x='within an hour'
    elif x == 4:
        x='within a few hours'
    elif x == 24:
        x='within a day'
    elif x == 48:
        x= 'a few days or more'
    elif x == 96:
        x= 'Not Response'
        
    return x

In [259]:
# Parsing csv file
import pandas as pd

seattle_file_path = 'seattle_data/listings.csv'
seattle_data = pd.read_csv(seattle_file_path)
# seattle_data.columns

In [274]:
# Make a new attribute
seattle_data_new = seattle_data.copy()
seattle_data_new['new_score_reviews'] = seattle_data['reviews_per_month'] * seattle_data['review_scores_rating'] / 10
# Fill missing value with zero for column
seattle_data_new['new_score_reviews'].fillna(0, inplace = True)
seattle_data_new.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,new_score_reviews
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07,38.665
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,f,NaN,WASHINGTON,f,strict,t,t,6,1.48,14.208
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,f,NaN,WASHINGTON,f,strict,f,f,2,1.15,11.155
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN,0.000
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,f,NaN,WASHINGTON,f,strict,f,f,1,0.89,8.188


In [226]:
# seattle_data_new.describe()

In [263]:
# Prediction target
y = seattle_data_new.new_score_reviews
# y.describe()

In [270]:
# Data Processing for training set

# Fill missing value with zero for numeric type attributes
seattle_data_new['price'].fillna(0, inplace = True)
seattle_data_new['cleaning_fee'].fillna(0, inplace = True)
seattle_data_new['minimum_nights'].fillna(1, inplace = True)

# Convert string to number
# Some attribute like host_response_time has string values: 'within a few hours', 'within an hour', 'within a day'...
# host_response_time

# host_is_superhost


# Drop useless columns
# useless_colmuns = ['id','host_id','host_listings_count','host_total_listings_count','latitude','longitude',
#                    'is_location_exact','square_feet','weekly_price','monthly_price','maximum_nights',
#                    'availability_30','availability_60','availability_90','number_of_reviews','review_scores_rating',
#                    'review_scores_accuracy','review_scores_cleanliness','review_scores_checkin',
#                    'review_scores_checkin','review_scores_communication','review_scores_location',
#                    'review_scores_value','calculated_host_listings_count','reviews_per_month',
#                    'require_guest_profile_picture','require_guest_phone_verification']
# seattle_data_new = seattle_data_new.drop(useless_colmuns, axis = 1)


In [280]:
# Features
# Needed for sure: 
# price, cleaning_fee, host_response_time, host_is_superhost, host_identity_verified, minimum_nights, 
# instant_bookable, cancellation_policy
# others: amenities
seattle_features = [
    'price', 
    'cleaning_fee', 
#     'host_response_time', 
#     'host_is_superhost',
#     'host_identity_verified',
    'minimum_nights',
#     'instant_bookable',
#     'cancellation_policy',

 ]
X = seattle_data_new[seattle_features]

# Statistic of the features
X.describe()

,minimum_nights
count,3818.000000
mean,2.369303
std,16.305902
min,1.000000
25%,1.000000
50%,2.000000
75%,2.000000
max,1000.000000


In [281]:
X.head()

,price,cleaning_fee,minimum_nights
0,$85.00,NaN,1
1,$150.00,$40.00,2
2,$975.00,$300.00,4
3,$100.00,NaN,1
4,$450.00,$125.00,1


In [282]:
# Baseline model
# Make prediction by mean or median

from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split

# split training and testing set. Training = 80%. Testing = 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# print(X_train.shape, y_train.shape)
# print(X_test.shape, y_test.shape)


(3054, 3) (3054,)
(764, 3) (764,)


In [286]:
# Fit a model
# Create a dummy regressor
baseline_model = DummyRegressor(strategy='mean')
# "Train" dummy regressor
baseline_model.fit(X_train, y_train)
# print(dummy_mean)

# Predict
# print("Baseline model predicitions using testing set ")
# print(X2.head())
# print("The predictions are:")
# print(baseline_model.predict(X_test))
print("The score is:")
print(baseline_model.score(X_test,y_test))

The score is:
-0.004057791170475111


In [231]:
# Making a model: DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
seattle_model = DecisionTreeRegressor(random_state=1)
seattle_model.fit(X,y);

print(seattle_model)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')


In [232]:
# Testing the model
# Not for real prediction. Need to use testing set to 
print("Making predicitions for the first 5 rows from training set:")
print(X.head())
print("The predictions are:")
print(seattle_model.predict(X.head()))

Making predicitions for the first 5 rows from training set:
   price  cleaning_fee  minimum_nights
0     85           0.0               1
1    150          40.0               2
2    975         300.0               4
3    100           0.0               1
4    450         125.0               1
The predictions are:
[26.4334     19.88642857 11.155       2.84894444  8.188     ]


In [233]:
# Making a prediction using testing set.
# Load testing set

seattle_file_path2 = 'seattle_data/testing.csv'
seattle_test_data = pd.read_csv(seattle_file_path2, encoding = 'utf-8')
# seattle_test_data.columns

In [234]:
seattle_test_data['new_score_reviews2'] = seattle_test_data['reviews_per_month'] * seattle_test_data['review_scores_rating'] / 10
# Fill missing value with zero for column
seattle_test_data['new_score_reviews2'].fillna(0, inplace = True)
seattle_test_data.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,new_score_reviews2
0,9432497,https://www.airbnb.com/rooms/9432497,2.020000e+13,1/4/16,Be at Home in Seattle,"Enjoy spacious living room, kitchen, sun room ...",NaN,"Enjoy spacious living room, kitchen, sun room ...",none,NaN,...,f,NaN,WASHINGTON,t,flexible,f,f,1,3.49,30.712
1,7931111,https://www.airbnb.com/rooms/7931111,2.020000e+13,1/4/16,"peaceful, cozy M-I-L apt.by Lake WA","This peaceful M-I-L basement apartment, with ...",To enter the space one descends a flight of st...,"This peaceful M-I-L basement apartment, with ...",none,This is a beautiful neighborhood with homes th...,...,f,NaN,WASHINGTON,f,flexible,f,f,1,2.21,22.100
2,5031357,https://www.airbnb.com/rooms/5031357,2.020000e+13,1/4/16,NE Seattle Private Entry Studio!,New! Low Rates for Mar/Apr! A private room and...,The private studio is cozy and comfortable in ...,New! Low Rates for Mar/Apr! A private room and...,none,Matthews Beach neighborhood is a super quiet a...,...,f,NaN,WASHINGTON,f,moderate,f,f,1,0.25,2.000
3,6603376,https://www.airbnb.com/rooms/6603376,2.020000e+13,1/4/16,Architectural Retreat near U of W,Bright and funky architectural home (Wendell L...,"Three Bedrooms, 2.5 bathrooms, and Bonus Sleep...",Bright and funky architectural home (Wendell L...,none,"The neighborhood is known as ""Matthews Beach""....",...,f,NaN,WASHINGTON,f,strict,f,f,1,4.40,43.560
4,3861673,https://www.airbnb.com/rooms/3861673,2.020000e+13,1/4/16,"Detached bedroom, bath in Ballard","This lovely, modern room and ensuite bath is d...",For those who would like more privacy than tha...,"This lovely, modern room and ensuite bath is d...",none,Ballard is known as one of the hippest neighbo...,...,f,NaN,WASHINGTON,f,moderate,f,f,1,10.61,105.039


In [235]:
# Data Processing for training set

# Fill missing value with zero for numeric type attributes
seattle_test_data['price'].fillna(0, inplace = True)
seattle_test_data['cleaning_fee'].fillna(0, inplace = True)
seattle_test_data['minimum_nights'].fillna(1, inplace = True)

# Convert string to number
# Some attribute like host_response_time has string values: 'within a few hours', 'within an hour', 'within a day'...
# host_response_time

# host_is_superhost


# Drop useless columns
seattle_test_data_new = seattle_test_data.drop(useless_colmuns, axis = 1)
# seattle_test_data_new.columns

In [236]:
# Features
seattle_features = [
    'price', 
    'cleaning_fee', 
#     'host_response_time', 
#     'host_is_superhost',
    'minimum_nights',
#     'availability_30', 
#     'review_scores_rating',
#     'instant_bookable',
#     'cancellation_policy',
#     'reviews_per_month',
#     'new_score_reviews2',
 ]
X2 = seattle_test_data[seattle_features]

In [250]:
# Predict by using previous model
print("Making predicitions:")
print(X2.head(10))
print("The predictions are:")
print(seattle_model.predict(X2.head(10)))

Making predicitions:
   price  cleaning_fee  minimum_nights
0     43           0.0               1
1     68          25.0               2
2     70          50.0               1
3    252         149.0               1
4     71           0.0               1
5     99           0.0               3
6    140         100.0               3
7    105          50.0               2
8    115          60.0               1
9     60           0.0               1
The predictions are:
[ 0.         47.724       0.          0.         18.00392308 30.478
  4.186       0.         22.601       9.58246667]
